# Модуль Г.  Разработка программного продукта

### 4.1 Предсказание целевой переменной 

In [34]:
xgb.max_depth = 6
xgb.random_state =1111
xgb.fit(x_train,y_train)

XGBClassifier(colsample_bytree=0.8, learning_rate=0.19, max_depth=6,
              n_estmats=22, nthread=4, objective='multi:softprob',
              random_state=1111, seed=27, subsample=0.9)

In [35]:
classification_predictions = xgb.predict(x_test)
probability_predictions = xgb.predict_proba(x_test)
# Распечатать количество двоичных прогнозов
print(pd.Series(classification_predictions).value_counts())
# Вывести первое значение из probability_predictions
print('Первыми предсказанными вероятностями являются: {}'.format(probability_predictions[0]))

9    12
2     1
dtype: int64
Первыми предсказанными вероятностями являются: [0.00424473 0.00350752 0.00573545 0.00239445 0.00204021 0.00916343
 0.00312372 0.96979046]


*Сохранение предсказаний для дальнейших использований*

In [36]:
from flask import render_template, request, jsonify
import flask
import numpy as np
import traceback
import pickle
import pandas as pd

In [37]:
import pickle
with open('predict.pkl','wb') as file:
    pickle.dump(probability_predictions, file)

### 4.2 Разработка прикладного решения

In [37]:
%%writefile movieclassifier_with_update/app.py
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import sqlite3
import os
import numpy as np
from vectorizer import vect

app = Flask(__name__)

cur_dir = os.path.dirname(__file__)
clf = pickle.load(open(os.path.join(cur_dir,
                 'pkl_objects',
                 'predict.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

def classify(document):
    label = {0: 'negative', 1: 'positive'}
    X = vect.transform([document])
    y = clf.predict(X)[0]
    proba = np.max(clf.predict_proba(X))
    return label[y], proba

def train(document, y):
    X = vect.transform([document])
    clf.partial_fit(X, [y])

def sqlite_entry(path, document, y):
    conn = sqlite3.connect(path)
    c = conn.cursor()
    c.execute("INSERT INTO review_db (review, sentiment, date)"\
    " VALUES (?, ?, DATETIME('now'))", (document, y))
    conn.commit()
    conn.close()

######## Flask
class ReviewForm(Form):
    moviereview = TextAreaField('',
                                [validators.DataRequired(),
                                validators.length(min=15)])

@app.route('/')
def index():
    form = ReviewForm(request.form)
    return render_template('reviewform.html', form=form)

@app.route('/results', methods=['POST'])
def results():
    form = ReviewForm(request.form)
    if request.method == 'POST' and form.validate():
        review = request.form['moviereview']
        y, proba = classify(review)
        return render_template('results.html',
                                content=review,
                                prediction=y,
                                probability=round(proba*100, 2))
    return render_template('reviewform.html', form=form)

@app.route('/thanks', methods=['POST'])
def feedback():
    feedback = request.form['feedback_button']
    review = request.form['review']
    prediction = request.form['prediction']

    inv_label = {'negative': 0, 'positive': 1}
    y = inv_label[prediction]
    if feedback == 'Incorrect':
        y = int(not(y))
    train(review, y)
    sqlite_entry(db, review, y)
    return render_template('thanks.html')

if __name__ == '__main__':
    app.run(debug=True)

Writing movieclassifier_with_update/app.py


In [39]:
%%writefile movieclassifier_with_update/templates/_formhelpers.html
{% macro render_field(field) %}
  <dt>{{ field.label }}
  <dd>{{ field(**kwargs)|safe }}
  {% if field.errors %}
    <ul class=errors>
    {% for error in field.errors %}
      <li>{{ error }}</li>
    {% endfor %}
    </ul>
  {% endif %}
  </dd>
{% endmacro %}

In [40]:
%%writefile movieclassifier_with_update/templates/results.html
<!doctype html>
<html>
  <head>
    <title>Прогнозирование номинаций по публикациям</title>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>

<h3>номинации:</h3>
<div>{{ content }}</div>

<h3>Прогноз:</h3>
<div>Этот публикация является <strong>{{ prediction }}</strong>
(с вероятностью:{{ probability }}%).</div>

<div id='button'>
<form action="/thanks" method="post">
        <input type=hidden value='{{ prediction }}' name='prediction'>
        <input type=hidden value='{{ content }}' name='review'>
    </form>
</div>

<div id='button'>
     <form action="/">
        <input type=submit value='Отправить публикацию'>
     </form>
</div>

  </body>
</html>

In [ ]:
%%writefile movieclassifier_with_update/templates/thanks.html
<!doctype html>
<html>
  <head>
    <title>Прогнозирование номинаций по публикациям</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>
<div id='button'>
        <form action="/">
        <input type=submit value='Отправить ещё публикацию'>
        </form>
</div>

  </body>
</html>

In [43]:
%%writefile movieclassifier_with_update/static/style.css
body{
    width:600px;
}

.button{
    padding-top: 20px;
}

In [42]:
%%writefile movieclassifier_with_update/update.py
import pickle
import sqlite3
import numpy as np
import os
from vectorizer import vect

def update_model(db_path, model, batch_size=10000):

    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute('SELECT * from review_db')

    results = c.fetchmany(batch_size)
    while results:
        data = np.array(results)
        X = data[:, 0]
        y = data[:, 1].astype(int)

        classes = np.array([0, 1])
        X_train = vect.transform(X)
        model.partial_fit(X_train, y, classes=classes)
        results = c.fetchmany(batch_size)

    conn.close()
    return model

cur_dir = os.path.dirname(__file__)

clf = pickle.load(open(os.path.join(cur_dir,
                  'pkl_objects',
                  'predict.pkl'), 'rb'))
db = os.path.join(cur_dir, 'reviews.sqlite')

clf = update_model(db_path=db, model=clf, batch_size=10000)

### 4.3 Подготовка отчета

*В этом модуле продолжается работа с данными, подготовленными в предыдущей сессии. Мне предстоит выполнить прогноз для тестовой выборки. Также необходимо было выполнить развертывание модели машинного обучения в рабочей среде в качестве API.*